In [ ]:
from osgeo import gdal, osr
import os
import numpy as np
import pandas as pd
import time
import glob
import datetime as dt

In [ ]:
def swath_georeference(in_file, target_dataset_name, out_file, x_res_out, y_res_out,out_geo_range=None):
    '''
    - `in_file`：输入文件的路径，这是一个包含卫星轨道数据的文件。
    - `target_dataset_name`：目标数据集的名称，这可是输入文件中的一个特定子集或层。
    - `out_file`：输出文件的路径，这是一个包含地理参考后的数据的新文件。
    - `x_res_out` 和 `y_res_out`：输出数据的空间分辨率，这决定了输出图像的每个像素代表的地理区域的大小。
    - `out_geo_range`：输出数据的地理范围，这是一个可选参数，如果提供，输出数据将被裁剪到这个范围。
    - `swath_georeference` 函数的目的是将卫星轨道数据转换为地理参考的数据，这样每个像素都可以对应到地球上的一个具体位置。
    '''    
    dataset = gdal.Open(in_file)
    if dataset == None:
        print('{} is not found or open.'.format(in_file))#转换失败
        return False
    subdatasets = dataset.GetSubDatasets()
    dataset_pos = 0
    target_pos = -1
    #找到目标数据集并返回索引号
    for subdataset in subdatasets:
        subdataset_name = subdataset[0]
        if subdataset_name.endswith(target_dataset_name):
            target_pos = dataset_pos
        dataset_pos += 1
    if target_pos != -1:
        target_dataset = dataset.GetSubDatasets()[target_pos][0]
        vrt_file = os.path.splitext(in_file)[0] + '.vrt'
        vrt_out = gdal.Translate(vrt_file, target_dataset,format='vrt', unscale='true')
        sr = osr.SpatialReference()
        sr.SetWellKnownGeogCS('EPSG:4326')#WGS-84坐标EPSG:4326
        georeferenced_data = gdal.Warp(out_file, vrt_out, multithread=True, outputBounds=out_geo_range,
                                       format='GTiff', geoloc=True, dstSRS=sr.ExportToWkt(),
                                       xRes=x_res_out, yRes=y_res_out, srcNodata=-9999,dstNodata=-9999,resampleAlg=gdal.GRIORA_NearestNeighbour,outputType=gdal.GDT_Float32,
                                       options=["TILED=YES", "COMPRESS=LZW"])
        if georeferenced_data == None:
            print('{}  is False.'.format(in_file))#转换失败
            os.remove(vrt_file)
        else:
            print('The georeference of {} is complete.'.format(in_file))#转换成功
            os.remove(vrt_file)
            pass
    else:
        print('{} has no the target dataset.'.format(in_file))
    # return True
    

In [ ]:
hdf=r"H:\C6.1\AQUA\2013\data\A2013001\MYD04_L2.A2013001.0505.061.2018044005624.hdf"
output_directory=r"E:\Orange\desktop\test"

#转换数据集名称
dataset_name = 'Deep_Blue_Aerosol_Optical_Depth_550_Land_Best_Estimate'
# dataset_name = 'Deep_Blue_Aerosol_Optical_Depth_550_Land'

geo_range = None  # 不定义图像范围
x_res = 0.1  # 分辨率
y_res = 0.1  # 分辨率

out_file = os.path.join(output_directory, os.path.splitext(os.path.basename(hdf))[0] + '_geo.tif')

swath_georeference(hdf, dataset_name,out_file,x_res, y_res,geo_range)
